In [ ]:
from database_tools.io.records import locate_valid_records

signals = ['PLETH', 'ABP']
min_length = 75000  # 10 min in samples (125 Hz)
n_segments = 50

valid_segs = locate_valid_records(signals, min_length, n_segments)

In [37]:
from database_tools.io.records import get_data_record

rcd = get_data_record('30/3000063/3000063_0016', record_type='waveforms')

In [39]:
rcd.sig_name

['II', 'ABP', 'PLETH']

In [55]:
set(rcd.fmt)

{'80'}

In [44]:
rcd.e_p_signal

In [ ]:
from wfdb.processing import find_local_peaks

In [1]:
class Signal:
    def __init__(self) -> None:
        self.names = ['ppg', 'abp']
        self.sigs = [[0, 1, 2, 3], [4, 5, 6, 7]]

class Record:
    def __init__(self) -> None:
        self.signals = Signal()
        [setattr(self, name, sig) for name, sig in zip(self.signals.names, self.signals.sigs)]

x = Record()

In [3]:
id(x.ppg)

140343697095872

In [5]:
id(x.signals.sigs[0])

140343697095872